In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

import cv2
from mss import mss
import time
import pyautogui

import ctypes
user32 = ctypes.windll.user32
screensize = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
screensize

(3440, 1440)

In [2]:
def process_directory(directory, label):
    features = []
    for filename in os.listdir(directory):
        if filename.endswith(".png"): 
            filepath = os.path.join(directory, filename)
#             print(filepath)
            features.append(preprocess(filepath))
    return features

def preprocess(filepath):
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
    height, width = image.shape
    return image.reshape(height*width,)

In [3]:
def train_wave():
    negative_samples = process_directory("negative-wave", 0)
    negative_samples = pd.DataFrame(np.array(negative_samples, dtype=np.float32))
    negative_samples["label"] = 0

    positive_samples = process_directory("positive-wave", 1)
    positive_samples = pd.DataFrame(np.array(positive_samples, dtype=np.float32))
    positive_samples["label"] = 1

    dataset = pd.concat([negative_samples, positive_samples])

    X = dataset.drop('label', axis=1)
    Y = dataset["label"]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    model = MLPClassifier(hidden_layer_sizes=(16, 16, 16, 16), max_iter=1000, alpha=1e-5, solver='adam', verbose=0)
    model.fit(X_train,y_train)
    predictions = model.predict(scaler.transform(X_test))

    f1score = f1_score(y_test, predictions).round(4)
    accuracy = accuracy_score(y_test, predictions).round(4)
    cm = confusion_matrix(y_test,predictions)

    print("F1Score: {}".format(f1score))
    print("Accuracy: {}".format(accuracy))
    print("Confusion Matrix:\n{}".format(cm))
    
    return model, scaler
wave_model, wave_scaler = train_wave()

F1Score: 0.9634
Accuracy: 0.9688
Confusion Matrix:
[[232   8]
 [  5 171]]


In [4]:
def train_drop():
    negative_samples = process_directory("negative-drop", 0)
    negative_samples = pd.DataFrame(np.array(negative_samples, dtype=np.float32))
    negative_samples["label"] = 0

    positive_samples = process_directory("positive-drop", 1)
    positive_samples = pd.DataFrame(np.array(positive_samples, dtype=np.float32))
    positive_samples["label"] = 1

    dataset = pd.concat([negative_samples, positive_samples])

    X = dataset.drop('label', axis=1)
    Y = dataset["label"]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    model = MLPClassifier(hidden_layer_sizes=(16, 16), max_iter=1000, alpha=1e-4, solver='adam', verbose=0)
    model.fit(X_train,y_train)
    predictions = model.predict(scaler.transform(X_test))

    f1score = f1_score(y_test, predictions).round(4)
    accuracy = accuracy_score(y_test, predictions).round(4)
    cm = confusion_matrix(y_test,predictions)

    print("F1Score: {}".format(f1score))
    print("Accuracy: {}".format(accuracy))
    print("Confusion Matrix:\n{}".format(cm))
    
    return model, scaler
drop_model, drop_scaler = train_drop()

F1Score: 1.0
Accuracy: 1.0
Confusion Matrix:
[[ 91   0]
 [  0 119]]


In [5]:
def grab_drops(sct):
#     monitor = {'top': 265, 'left': 750, 'width': 200, 'height': 35}
    drops = {'top': 265, 'left': 1000, 'width': 200, 'height': 35}
    
    # Get raw pixels from the screen, save it to a Numpy array
    img = np.array(sct.grab(drops))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
    
    #Display the picture in grayscale
    winname = "Drops"
    cv2.namedWindow(winname) 
    cv2.moveWindow(winname, 830, 1000) 
    cv2.imshow(winname, img)
    # FOR COLLECTING DATA
    # cv2.imwrite("d-{}.png".format(time.time()), img)
    return img
    
def grab_wave(sct):
    wave = {'top': 125, 'left': 1520, 'width': 130, 'height': 35}
    img = np.array(sct.grab(wave))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
    
    #Display the picture in grayscale
    winname = "Wave"
    cv2.namedWindow(winname) 
    cv2.moveWindow(winname, 1280, 1000) 
    cv2.imshow(winname, img)
    # FOR COLLECTING DATA
#     cv2.imwrite("w-{}.png".format(time.time()), img)
    return img

In [6]:
# # FOR COLLECTING DATA

# PREPARATION_LIMIT = 400
# WAVE_LIMIT = 100
# DROP_LIMIT = 50

# preparation_counter = 0
# wave_counter = 0
# drop_counter = 0
# no_drop_counter = 0

# state = "preparation"

# with mss() as sct:
#     # Part of the screen to capture
#     while "Screen capturing":
#         wave = grab_wave(sct)
#         drop = grab_drops(sct)

#         # Press "q" to quit
#         if cv2.waitKey(25) & 0xFF == ord("q"):
#             cv2.destroyAllWindows()
#             break

In [7]:
PREPARATION_LIMIT = 400
WAVE_LIMIT = 100
DROP_LIMIT = 50

preparation_counter = 0
wave_counter = 0
drop_counter = 0
no_drop_counter = 0

state = "preparation"

with mss() as sct:
    # Part of the screen to capture

    while "Screen capturing":
        last_time = time.time()
        
        if state == "preparation":
            print("[{}][{}/{}] Preparing for battle".format(state, preparation_counter, PREPARATION_LIMIT))
            preparation_counter += 1
            
            if preparation_counter > PREPARATION_LIMIT:
                pyautogui.click(x=930, y=300, interval=3)
                pyautogui.click(x=930, y=900)
                state = "battle"
                preparation_counter = 0
                
        wave = grab_wave(sct)
        
        if state == "battle":
            height, width = wave.shape
            wave_prediction = wave_model.predict(wave_scaler.transform([wave.reshape(height*width,)]))[0]

            if wave_prediction == 1:
                print("[{}][{}/{}] WAVE 3 Detected".format(state, wave_counter, WAVE_LIMIT))
                wave_counter += 1

            if wave_counter > WAVE_LIMIT:
                print("[{}][{}/{}] WAVE 3 Detected > {} times. Pressing 'ESC' now.".format(state, wave_counter, WAVE_LIMIT, WAVE_LIMIT))
                pyautogui.press('escape')
                state = "menu"
                wave_counter = 0
        
        drop = grab_drops(sct)

        if state == "menu":
            height, width = drop.shape
            drop_prediction = drop_model.predict(drop_scaler.transform([drop.reshape(height*width,)]))[0]

            if drop_prediction == 1:
                print("[{}][{}/{}] PURPLE DROP Detected.".format(state, drop_counter, DROP_LIMIT))
                drop_counter += 1

            if drop_counter > DROP_LIMIT:
                print("[{}][{}/{}] PURPLE DROP Detected > {} times.".format(state, drop_counter, DROP_LIMIT, DROP_LIMIT))
                state == "battle"
                drop_counter = 0
                exit()

            if drop_prediction == 0:
                print("[{}][{}/{}] NO PURPLE DROP".format(state, no_drop_counter, DROP_LIMIT))
                no_drop_counter += 1

            if no_drop_counter > DROP_LIMIT:
                print("[{}][{}/{}] NO PURPLE DROP. Quitting battle.".format(state, no_drop_counter, DROP_LIMIT))
                pyautogui.click(x=930, y=775, interval=2)
                pyautogui.click(x=930, y=775)
                state = "preparation"
                no_drop_counter = 0

#         print("fps: {}".format(1 / (time.time() - last_time)))

        # Press "q" to quit
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            break

[preparation][0/400] Preparing for battle
[preparation][1/400] Preparing for battle
[preparation][2/400] Preparing for battle
[preparation][3/400] Preparing for battle
[preparation][4/400] Preparing for battle
[preparation][5/400] Preparing for battle
[preparation][6/400] Preparing for battle
[preparation][7/400] Preparing for battle
[preparation][8/400] Preparing for battle
[preparation][9/400] Preparing for battle
[preparation][10/400] Preparing for battle
[preparation][11/400] Preparing for battle
[preparation][12/400] Preparing for battle
[preparation][13/400] Preparing for battle
[preparation][14/400] Preparing for battle
[preparation][15/400] Preparing for battle
[preparation][16/400] Preparing for battle
[preparation][17/400] Preparing for battle
[preparation][18/400] Preparing for battle
[preparation][19/400] Preparing for battle
[preparation][20/400] Preparing for battle
[preparation][21/400] Preparing for battle
[preparation][22/400] Preparing for battle
[preparation][23/400]

[preparation][189/400] Preparing for battle
[preparation][190/400] Preparing for battle
[preparation][191/400] Preparing for battle
[preparation][192/400] Preparing for battle
[preparation][193/400] Preparing for battle
[preparation][194/400] Preparing for battle
[preparation][195/400] Preparing for battle
[preparation][196/400] Preparing for battle
[preparation][197/400] Preparing for battle
[preparation][198/400] Preparing for battle
[preparation][199/400] Preparing for battle
[preparation][200/400] Preparing for battle
[preparation][201/400] Preparing for battle
[preparation][202/400] Preparing for battle
[preparation][203/400] Preparing for battle
[preparation][204/400] Preparing for battle
[preparation][205/400] Preparing for battle
[preparation][206/400] Preparing for battle
[preparation][207/400] Preparing for battle
[preparation][208/400] Preparing for battle
[preparation][209/400] Preparing for battle
[preparation][210/400] Preparing for battle
[preparation][211/400] Preparing

[preparation][377/400] Preparing for battle
[preparation][378/400] Preparing for battle
[preparation][379/400] Preparing for battle
[preparation][380/400] Preparing for battle
[preparation][381/400] Preparing for battle
[preparation][382/400] Preparing for battle
[preparation][383/400] Preparing for battle
[preparation][384/400] Preparing for battle
[preparation][385/400] Preparing for battle
[preparation][386/400] Preparing for battle
[preparation][387/400] Preparing for battle
[preparation][388/400] Preparing for battle
[preparation][389/400] Preparing for battle
[preparation][390/400] Preparing for battle
[preparation][391/400] Preparing for battle
[preparation][392/400] Preparing for battle
[preparation][393/400] Preparing for battle
[preparation][394/400] Preparing for battle
[preparation][395/400] Preparing for battle
[preparation][396/400] Preparing for battle
[preparation][397/400] Preparing for battle
[preparation][398/400] Preparing for battle
[preparation][399/400] Preparing

[preparation][51/400] Preparing for battle
[preparation][52/400] Preparing for battle
[preparation][53/400] Preparing for battle
[preparation][54/400] Preparing for battle
[preparation][55/400] Preparing for battle
[preparation][56/400] Preparing for battle
[preparation][57/400] Preparing for battle
[preparation][58/400] Preparing for battle
[preparation][59/400] Preparing for battle
[preparation][60/400] Preparing for battle
[preparation][61/400] Preparing for battle
[preparation][62/400] Preparing for battle
[preparation][63/400] Preparing for battle
[preparation][64/400] Preparing for battle
[preparation][65/400] Preparing for battle
[preparation][66/400] Preparing for battle
[preparation][67/400] Preparing for battle
[preparation][68/400] Preparing for battle
[preparation][69/400] Preparing for battle
[preparation][70/400] Preparing for battle
[preparation][71/400] Preparing for battle
[preparation][72/400] Preparing for battle
[preparation][73/400] Preparing for battle
[preparatio

[preparation][240/400] Preparing for battle
[preparation][241/400] Preparing for battle
[preparation][242/400] Preparing for battle
[preparation][243/400] Preparing for battle
[preparation][244/400] Preparing for battle
[preparation][245/400] Preparing for battle
[preparation][246/400] Preparing for battle
[preparation][247/400] Preparing for battle
[preparation][248/400] Preparing for battle
[preparation][249/400] Preparing for battle
[preparation][250/400] Preparing for battle
[preparation][251/400] Preparing for battle
[preparation][252/400] Preparing for battle
[preparation][253/400] Preparing for battle
[preparation][254/400] Preparing for battle
[preparation][255/400] Preparing for battle
[preparation][256/400] Preparing for battle
[preparation][257/400] Preparing for battle
[preparation][258/400] Preparing for battle
[preparation][259/400] Preparing for battle
[preparation][260/400] Preparing for battle
[preparation][261/400] Preparing for battle
[preparation][262/400] Preparing

[battle][35/100] WAVE 3 Detected
[battle][36/100] WAVE 3 Detected
[battle][37/100] WAVE 3 Detected
[battle][38/100] WAVE 3 Detected
[battle][39/100] WAVE 3 Detected
[battle][40/100] WAVE 3 Detected
[battle][41/100] WAVE 3 Detected
[battle][42/100] WAVE 3 Detected
[battle][43/100] WAVE 3 Detected
[battle][44/100] WAVE 3 Detected
[battle][45/100] WAVE 3 Detected
[battle][46/100] WAVE 3 Detected
[battle][47/100] WAVE 3 Detected
[battle][48/100] WAVE 3 Detected
[battle][49/100] WAVE 3 Detected
[battle][50/100] WAVE 3 Detected
[battle][51/100] WAVE 3 Detected
[battle][52/100] WAVE 3 Detected
[battle][53/100] WAVE 3 Detected
[battle][54/100] WAVE 3 Detected
[battle][55/100] WAVE 3 Detected
[battle][56/100] WAVE 3 Detected
[battle][57/100] WAVE 3 Detected
[battle][58/100] WAVE 3 Detected
[battle][59/100] WAVE 3 Detected
[battle][60/100] WAVE 3 Detected
[battle][61/100] WAVE 3 Detected
[battle][62/100] WAVE 3 Detected
[battle][63/100] WAVE 3 Detected
[battle][64/100] WAVE 3 Detected
[battle][6

[preparation][15/400] Preparing for battle
[preparation][16/400] Preparing for battle
[preparation][17/400] Preparing for battle
[preparation][18/400] Preparing for battle
[preparation][19/400] Preparing for battle
[preparation][20/400] Preparing for battle
[preparation][21/400] Preparing for battle
[preparation][22/400] Preparing for battle
[preparation][23/400] Preparing for battle
[preparation][24/400] Preparing for battle
[preparation][25/400] Preparing for battle
[preparation][26/400] Preparing for battle
[preparation][27/400] Preparing for battle
[preparation][28/400] Preparing for battle
[preparation][29/400] Preparing for battle
[preparation][30/400] Preparing for battle
[preparation][31/400] Preparing for battle
[preparation][32/400] Preparing for battle
[preparation][33/400] Preparing for battle
[preparation][34/400] Preparing for battle
[preparation][35/400] Preparing for battle
[preparation][36/400] Preparing for battle
[preparation][37/400] Preparing for battle
[preparatio